In [60]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import haversine as hs
from haversine import Unit
import numpy as np

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [61]:
# Load the housing sale prices data.
houses_df = pd.read_csv('sale_price.csv')

# Add columns for assigned elementary, middle and high schools.
houses_df['elem'] = np.nan
houses_df['middle'] = np.nan
houses_df['high'] = np.nan

houses_df.head()

,Unnamed: 0,zpid,zipcode,city,state,latitude,longitude,price,dateSold,bathrooms,...,isPremierBuilder,isZillowOwned,currency,country,taxAssessedValue,lotAreaValue,lotAreaUnit,elem,middle,high
0,0,37151520,20854,Potomac,MD,39.043986,-77.256456,800000.0,1625641200000,4.0,...,False,False,USD,USA,1081300.0,5.10,acres,NaN,NaN,NaN
1,1,246554034,20814,Bethesda,MD,39.022005,-77.107586,800000.0,1617606000000,1.0,...,False,False,USD,USA,507900.0,0.25,acres,NaN,NaN,NaN
2,2,37181535,20816,Bethesda,MD,38.958795,-77.111890,800000.0,1624604400000,3.0,...,False,False,USD,USA,721567.0,8750.00,sqft,NaN,NaN,NaN
3,3,37182564,20817,Bethesda,MD,38.974071,-77.131494,800000.0,1604563200000,3.0,...,False,False,USD,USA,697767.0,9148.00,sqft,NaN,NaN,NaN
4,4,37096069,20854,Potomac,MD,39.041634,-77.172128,800000.0,1620370800000,3.0,...,False,False,USD,USA,611000.0,9811.00,sqft,NaN,NaN,NaN


In [62]:
# Utility function to return a list of clean school names.
def get_schools(schools_df):
    
    ret = []
    
    # Access all schools and put them in a list.
    for s in schools_df['School'].tolist():
        
        # Split the string.
        arr = s.split()
        
        # If the last word is 'School', drop it.
        if arr[len(arr)-1].lower() == 'school':
            arr.pop()
            
        # Convert all words to lowercase.
        for i in range(0, len(arr)):
            arr[i] = arr[i].lower()
            
        ret.append(' '.join(arr))
    
    return ret

In [63]:
# Utility function to return a "cleaned school" name.
def clean_school_name(s):
    
    arr = s.split()
    
    # If the last word is 'School', drop it.
    if arr[len(arr)-1].lower() == 'school':
        arr.pop()
        
    # Convert all words to lowercase.
    for i in range(0, len(arr)):
        arr[i] = arr[i].lower()
        
    return ' '.join(arr)

In [64]:
# Load the school ratings data.
schools_df = pd.read_csv('mcps_school_ratings.csv')

# Get a list of "cleaned" elementary school names.
elem_schools = get_schools(schools_df.loc[schools_df['Type'] == 'E'])

# Get a list of "cleaned" middle school names.
middle_schools = get_schools(schools_df.loc[schools_df['Type'] == 'M'])

# Get a list of "cleaned" high school names.
high_schools = get_schools(schools_df.loc[schools_df['Type'] == 'H'])

In [65]:
# Utility function to assign primary school to the given address based on geographical proximity.
def get_assigned_primary_school(gmap_results, loc):
    
    elem_distance = 100000.0
    assigned_elem = None

    for s in gmap_results['results']:
                
        s_name = clean_school_name(s['name'])
        s_lat = s['geometry']['location']['lat']
        s_lng = s['geometry']['location']['lng']

    
        # Check if this name matches to a potential elementary school name in MCPS.
        if s_name in elem_schools:
        
            dist = hs.haversine(loc,
                                (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < elem_distance:
                elem_distance = dist
                assigned_elem = s_name
                #print (f'Found elementary school {assigned_elem} with dist {elem_distance}')
    
    return assigned_elem

In [66]:
# Utility function to assign middle and high schools to the given address based on geographical proximity.
def get_assigned_secondary_schools(gmap_results, loc):
    
    middle_distance = 100000.0
    high_distance = 100000.0
    
    assigned_middle = None
    assigned_high = None

    for s in gmap_results['results']:
            
        s_name = clean_school_name(s['name'])
        s_lat = s['geometry']['location']['lat']
        s_lng = s['geometry']['location']['lng']

    
        # Check if this name matches to a potential middle school name in MCPS.
        if s_name in middle_schools:
        
            dist = hs.haversine(loc,
                                (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < middle_distance:
                middle_distance = dist
                assigned_middle = s_name
                #print (f'Found middle school {assigned_middle} with dist {middle_distance}')
            
        # Check if this name matches to a potential high school name in MCPS.
        elif s_name in high_schools:
        
            dist = hs.haversine((39.043986, -77.256456),
                               (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < high_distance:
                high_distance = dist
                assigned_high = s_name
                #print (f'Found high school {assigned_high} with dist {high_distance}')
    
    return (assigned_middle, assigned_high)

In [72]:
####################### Assign Elementary School. #############################################
###### IMPORTANT: Google API may ratelimit, so it may be necessary to kill the process. ######
###### If this happens, export the csv and start from the next index onwards. ################
###### Once ran, do NOT run this code again causing thousands of gmap API calls. #############

# Set parameters to search for primary schools within 10000 meters.
params = {
    "radius": 10000,
    "type": "primary_school",
    "key": gkey
}

# Construct the baseurl.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in houses_df.iterrows():
    
    # Add lat, lng to the parameters.
    #params['location'] = f"{row['latitude']},{row['longitude']}"
    
    # Make request and retrieve the JSON data from the search. 
    #try:
        #potential_elem_schools = requests.get(base_url, params=params).json()
        #assigned_elem_school = get_assigned_primary_school(potential_elem_schools, 
        #                                                  (row['latitude'], row['longitude'])
        #                                                  )
        
        # Assign school.
        #if assigned_elem_school:
        #    houses_df.loc[index, 'elem'] = assigned_elem_school
    
    #except Exception as e:
    #    print (f"Index: {index}, Zpid: {row['zpid']}, response message {schools['message']}")
        
    print(f'Index: {index}')

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29
Index: 30
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Index: 49
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Index: 81
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97
Index: 98
Index: 99
Index: 100

Index: 755
Index: 756
Index: 757
Index: 758
Index: 759
Index: 760
Index: 761
Index: 762
Index: 763
Index: 764
Index: 765
Index: 766
Index: 767
Index: 768
Index: 769
Index: 770
Index: 771
Index: 772
Index: 773
Index: 774
Index: 775
Index: 776
Index: 777
Index: 778
Index: 779
Index: 780
Index: 781
Index: 782
Index: 783
Index: 784
Index: 785
Index: 786
Index: 787
Index: 788
Index: 789
Index: 790
Index: 791
Index: 792
Index: 793
Index: 794
Index: 795
Index: 796
Index: 797
Index: 798
Index: 799
Index: 800
Index: 801
Index: 802
Index: 803
Index: 804
Index: 805
Index: 806
Index: 807
Index: 808
Index: 809
Index: 810
Index: 811
Index: 812
Index: 813
Index: 814
Index: 815
Index: 816
Index: 817
Index: 818
Index: 819
Index: 820
Index: 821
Index: 822
Index: 823
Index: 824
Index: 825
Index: 826
Index: 827
Index: 828
Index: 829
Index: 830
Index: 831
Index: 832
Index: 833
Index: 834
Index: 835
Index: 836
Index: 837
Index: 838
Index: 839
Index: 840
Index: 841
Index: 842
Index: 843
Index: 844
Index: 845

Index: 1459
Index: 1460
Index: 1461
Index: 1462
Index: 1463
Index: 1464
Index: 1465
Index: 1466
Index: 1467
Index: 1468
Index: 1469
Index: 1470
Index: 1471
Index: 1472
Index: 1473
Index: 1474
Index: 1475
Index: 1476
Index: 1477
Index: 1478
Index: 1479
Index: 1480
Index: 1481
Index: 1482
Index: 1483
Index: 1484
Index: 1485
Index: 1486
Index: 1487
Index: 1488
Index: 1489
Index: 1490
Index: 1491
Index: 1492
Index: 1493
Index: 1494
Index: 1495
Index: 1496
Index: 1497
Index: 1498
Index: 1499
Index: 1500
Index: 1501
Index: 1502
Index: 1503
Index: 1504
Index: 1505
Index: 1506
Index: 1507
Index: 1508
Index: 1509
Index: 1510
Index: 1511
Index: 1512
Index: 1513
Index: 1514
Index: 1515
Index: 1516
Index: 1517
Index: 1518
Index: 1519
Index: 1520
Index: 1521
Index: 1522
Index: 1523
Index: 1524
Index: 1525
Index: 1526
Index: 1527
Index: 1528
Index: 1529
Index: 1530
Index: 1531
Index: 1532
Index: 1533
Index: 1534
Index: 1535
Index: 1536
Index: 1537
Index: 1538
Index: 1539
Index: 1540
Index: 1541
Inde

Index: 2142
Index: 2143
Index: 2144
Index: 2145
Index: 2146
Index: 2147
Index: 2148
Index: 2149
Index: 2150
Index: 2151
Index: 2152
Index: 2153
Index: 2154
Index: 2155
Index: 2156
Index: 2157
Index: 2158
Index: 2159
Index: 2160
Index: 2161
Index: 2162
Index: 2163
Index: 2164
Index: 2165
Index: 2166
Index: 2167
Index: 2168
Index: 2169
Index: 2170
Index: 2171
Index: 2172
Index: 2173
Index: 2174
Index: 2175
Index: 2176
Index: 2177
Index: 2178
Index: 2179
Index: 2180
Index: 2181
Index: 2182
Index: 2183
Index: 2184
Index: 2185
Index: 2186
Index: 2187
Index: 2188
Index: 2189
Index: 2190
Index: 2191
Index: 2192
Index: 2193
Index: 2194
Index: 2195
Index: 2196
Index: 2197
Index: 2198
Index: 2199
Index: 2200
Index: 2201
Index: 2202
Index: 2203
Index: 2204
Index: 2205
Index: 2206
Index: 2207
Index: 2208
Index: 2209
Index: 2210
Index: 2211
Index: 2212
Index: 2213
Index: 2214
Index: 2215
Index: 2216
Index: 2217
Index: 2218
Index: 2219
Index: 2220
Index: 2221
Index: 2222
Index: 2223
Index: 2224
Inde

Index: 2825
Index: 2826
Index: 2827
Index: 2828
Index: 2829
Index: 2830
Index: 2831
Index: 2832
Index: 2833
Index: 2834
Index: 2835
Index: 2836
Index: 2837
Index: 2838
Index: 2839
Index: 2840
Index: 2841
Index: 2842
Index: 2843
Index: 2844
Index: 2845
Index: 2846
Index: 2847
Index: 2848
Index: 2849
Index: 2850
Index: 2851
Index: 2852
Index: 2853
Index: 2854
Index: 2855
Index: 2856
Index: 2857
Index: 2858
Index: 2859
Index: 2860
Index: 2861
Index: 2862
Index: 2863
Index: 2864
Index: 2865
Index: 2866
Index: 2867
Index: 2868
Index: 2869
Index: 2870
Index: 2871
Index: 2872
Index: 2873
Index: 2874
Index: 2875
Index: 2876
Index: 2877
Index: 2878
Index: 2879
Index: 2880
Index: 2881
Index: 2882
Index: 2883
Index: 2884
Index: 2885
Index: 2886
Index: 2887
Index: 2888
Index: 2889
Index: 2890
Index: 2891
Index: 2892
Index: 2893
Index: 2894
Index: 2895
Index: 2896
Index: 2897
Index: 2898
Index: 2899
Index: 2900
Index: 2901
Index: 2902
Index: 2903
Index: 2904
Index: 2905
Index: 2906
Index: 2907
Inde

...

Index: 8294
Index: 8295
Index: 8296
Index: 8297
Index: 8298
Index: 8299
Index: 8300
Index: 8301
Index: 8302
Index: 8303
Index: 8304
Index: 8305
Index: 8306
Index: 8307
Index: 8308
Index: 8309
Index: 8310
Index: 8311
Index: 8312
Index: 8313
Index: 8314
Index: 8315
Index: 8316
Index: 8317
Index: 8318
Index: 8319
Index: 8320
Index: 8321
Index: 8322
Index: 8323
Index: 8324
Index: 8325
Index: 8326
Index: 8327
Index: 8328
Index: 8329
Index: 8330
Index: 8331
Index: 8332
Index: 8333
Index: 8334
Index: 8335
Index: 8336
Index: 8337
Index: 8338
Index: 8339
Index: 8340
Index: 8341
Index: 8342
Index: 8343
Index: 8344
Index: 8345
Index: 8346
Index: 8347
Index: 8348
Index: 8349
Index: 8350
Index: 8351
Index: 8352
Index: 8353
Index: 8354
Index: 8355
Index: 8356
Index: 8357
Index: 8358
Index: 8359
Index: 8360
Index: 8361
Index: 8362
Index: 8363
Index: 8364
Index: 8365
Index: 8366
Index: 8367
Index: 8368
Index: 8369
Index: 8370
Index: 8371
Index: 8372
Index: 8373
Index: 8374
Index: 8375
Index: 8376
Inde

Index: 8978
Index: 8979
Index: 8980
Index: 8981
Index: 8982
Index: 8983
Index: 8984
Index: 8985
Index: 8986
Index: 8987
Index: 8988
Index: 8989
Index: 8990
Index: 8991
Index: 8992
Index: 8993
Index: 8994
Index: 8995
Index: 8996
Index: 8997
Index: 8998
Index: 8999
Index: 9000
Index: 9001
Index: 9002
Index: 9003
Index: 9004
Index: 9005
Index: 9006
Index: 9007
Index: 9008
Index: 9009
Index: 9010
Index: 9011
Index: 9012
Index: 9013
Index: 9014
Index: 9015
Index: 9016
Index: 9017
Index: 9018
Index: 9019
Index: 9020
Index: 9021
Index: 9022
Index: 9023
Index: 9024
Index: 9025
Index: 9026
Index: 9027
Index: 9028
Index: 9029
Index: 9030
Index: 9031
Index: 9032
Index: 9033
Index: 9034
Index: 9035
Index: 9036
Index: 9037
Index: 9038
Index: 9039
Index: 9040
Index: 9041
Index: 9042
Index: 9043
Index: 9044
Index: 9045
Index: 9046
Index: 9047
Index: 9048
Index: 9049
Index: 9050
Index: 9051
Index: 9052
Index: 9053
Index: 9054
Index: 9055
Index: 9056
Index: 9057
Index: 9058
Index: 9059
Index: 9060
Inde

Index: 9661
Index: 9662
Index: 9663
Index: 9664
Index: 9665
Index: 9666
Index: 9667
Index: 9668
Index: 9669
Index: 9670
Index: 9671
Index: 9672
Index: 9673
Index: 9674
Index: 9675
Index: 9676
Index: 9677
Index: 9678
Index: 9679
Index: 9680
Index: 9681
Index: 9682
Index: 9683
Index: 9684
Index: 9685
Index: 9686
Index: 9687
Index: 9688
Index: 9689
Index: 9690
Index: 9691
Index: 9692
Index: 9693
Index: 9694
Index: 9695
Index: 9696
Index: 9697
Index: 9698
Index: 9699
Index: 9700
Index: 9701
Index: 9702
Index: 9703
Index: 9704
Index: 9705
Index: 9706
Index: 9707
Index: 9708
Index: 9709
Index: 9710
Index: 9711
Index: 9712
Index: 9713
Index: 9714
Index: 9715
Index: 9716
Index: 9717
Index: 9718
Index: 9719
Index: 9720
Index: 9721
Index: 9722
Index: 9723
Index: 9724
Index: 9725
Index: 9726
Index: 9727
Index: 9728
Index: 9729
Index: 9730
Index: 9731
Index: 9732
Index: 9733
Index: 9734
Index: 9735
Index: 9736
Index: 9737
Index: 9738
Index: 9739
Index: 9740
Index: 9741
Index: 9742
Index: 9743
Inde

Index: 10318
Index: 10319
Index: 10320
Index: 10321
Index: 10322
Index: 10323
Index: 10324
Index: 10325
Index: 10326
Index: 10327
Index: 10328
Index: 10329
Index: 10330
Index: 10331
Index: 10332
Index: 10333
Index: 10334
Index: 10335
Index: 10336
Index: 10337
Index: 10338
Index: 10339
Index: 10340
Index: 10341
Index: 10342
Index: 10343
Index: 10344
Index: 10345
Index: 10346
Index: 10347
Index: 10348
Index: 10349
Index: 10350
Index: 10351
Index: 10352
Index: 10353
Index: 10354
Index: 10355
Index: 10356
Index: 10357
Index: 10358
Index: 10359
Index: 10360
Index: 10361
Index: 10362
Index: 10363
Index: 10364
Index: 10365
Index: 10366
Index: 10367
Index: 10368
Index: 10369
Index: 10370
Index: 10371
Index: 10372
Index: 10373
Index: 10374
Index: 10375
Index: 10376
Index: 10377
Index: 10378
Index: 10379
Index: 10380
Index: 10381
Index: 10382
Index: 10383
Index: 10384
Index: 10385
Index: 10386
Index: 10387
Index: 10388
Index: 10389
Index: 10390
Index: 10391
Index: 10392
Index: 10393
Index: 10394

KeyboardInterrupt: 

In [75]:
####################### Assign Middle and High Schools. #############################################
###### IMPORTANT: Google API may ratelimit, so it may be necessary to kill the process. ######
###### If this happens, export the csv and start from the next index onwards. ################
###### Once ran, do NOT run this code again causing thousands of gmap API calls. #############

# Set parameters to search for primary schools within 10000 meters.
params = {
    "radius": 10000,
    "type": "secondary_school",
    "key": gkey
}

# Construct the baseurl.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in houses_df.iterrows():
    
    # Add lat, lng to the parameters.
    #params['location'] = f"{row['latitude']},{row['longitude']}"
    
    # Make request and retrieve the JSON data from the search. 
    #try:
        #potential_sec_schools = requests.get(base_url, params=params).json()
        #assigned_sec_schools = get_assigned_secondary_schools(potential_sec_schools, 
        #                                                    (row['latitude'], row['longitude'])
        #                                                    )
        
        # Assign schools.
        #if assigned_sec_schools[0]:
        #    houses_df.loc[index, 'middle'] = assigned_sec_schools[0]
        #if assigned_sec_schools[1]:
        #    houses_df.loc[index, 'high'] = assigned_sec_schools[1]
    
    #except Exception as e:
    #    print (f"Index: {index}, Zpid: {row['zpid']}, response message {schools['message']}")
        
    #print(f'Index: {index}')

############################ Must stay commented out in order not to overwrite. ##############
####### houses_df.to_csv('house_info_school_assignments.csv')
##############################################################################################

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29
Index: 30
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Index: 49
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Index: 81
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97
Index: 98
Index: 99
Index: 100

Index: 755
Index: 756
Index: 757
Index: 758
Index: 759
Index: 760
Index: 761
Index: 762
Index: 763
Index: 764
Index: 765
Index: 766
Index: 767
Index: 768
Index: 769
Index: 770
Index: 771
Index: 772
Index: 773
Index: 774
Index: 775
Index: 776
Index: 777
Index: 778
Index: 779
Index: 780
Index: 781
Index: 782
Index: 783
Index: 784
Index: 785
Index: 786
Index: 787
Index: 788
Index: 789
Index: 790
Index: 791
Index: 792
Index: 793
Index: 794
Index: 795
Index: 796
Index: 797
Index: 798
Index: 799
Index: 800
Index: 801
Index: 802
Index: 803
Index: 804
Index: 805
Index: 806
Index: 807
Index: 808
Index: 809
Index: 810
Index: 811
Index: 812
Index: 813
Index: 814
Index: 815
Index: 816
Index: 817
Index: 818
Index: 819
Index: 820
Index: 821
Index: 822
Index: 823
Index: 824
Index: 825
Index: 826
Index: 827
Index: 828
Index: 829
Index: 830
Index: 831
Index: 832
Index: 833
Index: 834
Index: 835
Index: 836
Index: 837
Index: 838
Index: 839
Index: 840
Index: 841
Index: 842
Index: 843
Index: 844
Index: 845

KeyboardInterrupt: 

In [87]:
# Load data again and remove all addresses with at least one unassigned school. Sava again.
houses_sch_assigned_df = pd.read_csv('house_info_school_assignments.csv')

print(f'There are {len(houses_sch_assigned_df)} records to begin with.')

houses_sch_assigned_df['elem'].replace('', np.nan, inplace=True)
houses_sch_assigned_df['middle'].replace('', np.nan, inplace=True)
houses_sch_assigned_df['high'].replace('', np.nan, inplace=True)
houses_sch_assigned_df.dropna(subset=['elem',
                                      'middle',
                                      'high'], 
                                      inplace=True)

print(f'There are {len(houses_sch_assigned_df)} records after dropping houses with at least one unassigned school.')

houses_sch_assigned_df.to_csv('house_info_school_assignments_dropna.csv')

There are 15818 records to begin with.
There are 13983 records after dropping houses with at least one unassigned school.
